<a href="https://colab.research.google.com/github/dltmddbs100/KSIC_Automation/blob/main/TF_roberta_large_skf%26ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome! 
the 1st solution of National Statistical Office Artificial Intelligence Contest.  
This notebook is a TensorFlow Notebook for Flat Roberta-Large Model.  
It uses  
- Colab TPU, You have to set your runtime TPU  
- Backbone: Klue/Roberta-Large
- mixed precision: bfloat16 for TPU

# Prepare basic environment  
- Colab environment doesn't have **Transformer** framework, pip install is easy way to install.  
- **Datasets**, is another library of huggingface.  
This could help Data handling for tensorflow or pytorch.  
- We can not offer the data we used for competition.
you have to get the data from https://data.kostat.go.kr/sbchome/index.do  

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install transformers sentencepiece spacy
!pip install Datasets
# preparing data
!unzip /content/drive/MyDrive/data_center_league/data/data_center_prize.zip 

Mounted at /content/drive
     |████████████████████████████████| 4.0 MB 5.2 MB/s 
     |████████████████████████████████| 1.2 MB 72.6 MB/s 
     |████████████████████████████████| 596 kB 71.5 MB/s 
     |████████████████████████████████| 6.6 MB 72.6 MB/s 
     |████████████████████████████████| 77 kB 6.6 MB/s 
     |████████████████████████████████| 895 kB 81.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 79.9 MB 1.1 MB/s 
     |████████████████████████████████| 325 kB 5.1 MB/s 
     |████████████████████████████████| 1.1 MB 71.6 MB/s 
     |████████████████████████████████| 136 kB 87.8 MB/s 
     |████████████████████████████████| 212 kB 75.7 MB/s 
     |████████████████████████████████| 127 kB 26.1 MB/s 
     |████████████████████████████████| 94 kB 84 kB/s 
     |████████████████████████████████| 271 kB 85.2 MB/s 
     |██████████████

# Import

In [ ]:
import sys
import os
from glob import glob
import warnings
from tqdm import tqdm

from transformers import TFAutoModel, AutoConfig, AutoTokenizer, AutoModel, TFAutoModelForSequenceClassification
from datasets import Dataset
import tensorflow.keras.backend as K
import tensorflow as tf
import torch

from sklearn import metrics
from sklearn import preprocessing
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold
import pandas as pd, numpy as np

tqdm.pandas()
warnings.filterwarnings("ignore")
tf.config.optimizer.set_experimental_options({"auto_mixed_precision": True})
tf.keras.mixed_precision.set_global_policy('mixed_bfloat16')

tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
strategy = tf.distribute.TPUStrategy(tpu)

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.43.68.202:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.43.68.202:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [ ]:
# set data path
train_csv_path = '/content/drive/MyDrive/data_center_league/AI-based_industry_classification/kerry/data/train_final_spacing.csv'
test_csv_path = '/content/drive/MyDrive/data_center_league/AI-based_industry_classification/kerry/data/test_final_spacing.csv'

# Data Preprocess

Preprocessed version of the data, has NAN token.   
But in this Model is going to assume that each features(3) are one sentence.  that's why NAN token is replaced.  
**text_obj, text_mthd, text_deal** are going to be a one sequence 
and this Model going to predict digit_3 directly.  


In [ ]:
train = pd.read_csv(train_csv_path)
test = pd.read_csv(test_csv_path)

In [ ]:
def strip_sentence(df_element):
    stripped = df_element.strip()
    return stripped
text_df = train[["text_obj","text_mthd","text_deal",]]
text_df.fillna("",inplace=True)
text_df.text_obj = text_df.text_obj.replace("[NAN]","")
text_df.text_mthd = text_df.text_mthd.replace("[NAN]","")
text_df.text_deal = text_df.text_deal.replace("[NAN]","")
text_df["sentence"] = text_df.text_obj + " " + text_df.text_mthd + " " + text_df.text_deal
text_df["strip_sentence"] = text_df.sentence.progress_apply(strip_sentence)
text_df["label"] = train.digit_3
text_df.drop(['text_obj','text_mthd','text_deal','sentence'],axis=1,inplace=True)
text_df.drop_duplicates(["strip_sentence","label"],inplace=True)
text_df.drop_duplicates("strip_sentence",inplace=True)

100%|██████████| 656231/656231 [00:00<00:00, 831413.65it/s]


# Model Choice

You can use another model, But klue/roberta-large from huggingface modelhub made best performance.  
save model in local storage to build model in TPU context.


In [ ]:
MAX_LEN = 35
MODEL_NAME = "klue/roberta-large"
MODEL_FILE_NAME = "klue-roberta-large"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
config = AutoConfig.from_pretrained(MODEL_NAME)
backbone = AutoModel.from_pretrained(MODEL_NAME,config=config)
tokenizer.save_pretrained('model')
config.save_pretrained('model')
backbone.save_pretrained('model')

Downloading:   0%|          | 0.00/375 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/734k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/173 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/547 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.decoder.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it f

# Tokenizing

Huggingface dataset works like python dictionary but works like tensorflow dataset as well.  
Map method with lambda function tokenize every sentence  
tokens = input_ids  
attention = attention_masks  

In [ ]:
text_ds = Dataset.from_pandas(text_df[["strip_sentence","label"]])
text_ds = text_ds.map(lambda x: tokenizer.encode_plus(x["strip_sentence"],max_length=MAX_LEN,padding="max_length",truncation=True,return_tensors="np"),)
text_ds = text_ds.remove_columns("strip_sentence")
train_tokens = np.array(text_ds['input_ids']).squeeze(1)
train_attentions = np.array(text_ds['attention_mask']).squeeze(1)

  0%|          | 0/649568 [00:00<?, ?ex/s]

# Label Encoding

This is Multiclass classification Task.  
225 classes are going to be changed with categorical method

In [ ]:
label_encoder = preprocessing.LabelEncoder()
label_encoder.fit(text_df.label.to_numpy(dtype=np.int32))
encoded_label = label_encoder.transform(text_df.label.to_numpy(dtype=np.int32))
encoded_label = encoded_label.astype(np.int32)
targets = tf.keras.utils.to_categorical(encoded_label)

# Token Save

###Tokenizing The data takes some time.  
###Once you save your tokens, you don't need to tokenize again


In [ ]:
token_target_path = '/content/drive/MyDrive/data_center_league/AI-based_industry_classification/Gu/[NAN]rob-large-no-duplicate-5fold/token-target/'

In [ ]:
np.save(f"{token_target_path}train_tokens-{MAX_LEN}.npy",train_tokens)
np.save(f"{token_target_path}train_attentions-{MAX_LEN}.npy",train_attentions)
np.save(f"{token_target_path}targets-{MAX_LEN}.npy",targets)

# Token Load

In [ ]:
train_tokens = np.load(f"{token_target_path}train_tokens-{MAX_LEN}.npy")
train_attentions = np.load(f"{token_target_path}train_attentions-{MAX_LEN}.npy")
targets = np.load(f"{token_target_path}targets-{MAX_LEN}.npy")

# Model setting

In [ ]:
AUTO = tf.data.AUTOTUNE
def batch_dataset(ds):
    return ds.repeat().shuffle(2048).batch(BATCH_SIZE,drop_remainder=True).prefetch(AUTO) 

def build_model():
    path = '/content/model/'
    tokens = tf.keras.layers.Input(shape = (MAX_LEN,), name = 'input_ids', dtype = tf.int32)
    attention = tf.keras.layers.Input(shape = (MAX_LEN,), name = 'attention_mask', dtype = tf.int32)
    config = AutoConfig.from_pretrained(path+'/config.json')
    backbone = TFAutoModel.from_pretrained(path+'pytorch_model.bin',from_pt=True,config=config)
    x = backbone(tokens, attention_mask = attention)
    x = tf.keras.layers.Dense(256,activation="relu")(x[1])
    x = tf.keras.layers.Dense(225, activation='softmax', dtype='float32',name="classifier")(x)
    model = tf.keras.Model(inputs = [tokens, attention], outputs = x)
    model.compile(optimizer = tf.keras.optimizers.Adam(lr = 1e-4,), 
              loss = tf.keras.losses.CategoricalCrossentropy(),
              metrics = tf.keras.metrics.CategoricalAccuracy(),
              steps_per_execution=115)
    
    return model

# Model Hyper Parameters

TPU has 8cores.  
Each core`s batch size is 64, Global batch size is 512  


In [ ]:
RUN_TRAINING = True
BATCH_SIZE = 64 * 8
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=2)
save_model_path = '/content/test_model_path/'
LRS_A = [3e-5 for i in range(1)]
LRS_B = [2e-5 for i in range(1)]
LRS_C = [1e-5 for i in range(1)]
LRS_D = [5e-6 for i in range(1)]
LRS = LRS_A + LRS_B + LRS_C + LRS_D
EPOCH = len(LRS)
def lrfn(epoch): return LRS[epoch]

# Model Train

We used same fold, random_state for average ensemble.

In [ ]:
%%time
for fold, (train_idx, valid_idx) in enumerate(StratifiedKFold(n_splits = 5, shuffle=True, random_state = 1514).split(train_tokens, text_df.label)):

    # initialize variables in tpu
    tf.tpu.experimental.initialize_tpu_system(tpu)

    # each fold it makes new dataset, new model for self average ensemble
    input_dic = {"input_ids":tf.convert_to_tensor(train_tokens[train_idx])}
    input_dic["attention_mask"] = tf.convert_to_tensor(train_attentions[train_idx])
    train_ds = tf.data.Dataset.from_tensor_slices((input_dic,targets[train_idx,]))
    train_ds = batch_dataset(train_ds)
    val_input_dic = {"input_ids":tf.convert_to_tensor(train_tokens[valid_idx,])}
    val_input_dic['attention_mask'] = tf.convert_to_tensor(train_attentions[valid_idx,])
    val_ds = tf.data.Dataset.from_tensor_slices((val_input_dic, targets[valid_idx,]))
    val_ds = batch_dataset(val_ds)
    train_ds = strategy.experimental_distribute_dataset(train_ds)
    val_ds = strategy.experimental_distribute_dataset(val_ds)
    print('#'*25)
    print('### FOLD %i' % (fold))
    print('#'*25)

    K.clear_session()

    with strategy.scope():
        model = build_model()

    if RUN_TRAINING: 
        history = model.fit(train_ds,
                    validation_data = val_ds,
                    steps_per_epoch = int(train_idx.size/BATCH_SIZE),
                        validation_steps = int(valid_idx.size/BATCH_SIZE),
                    callbacks = [es,tf.keras.callbacks.LearningRateScheduler(lrfn, verbose = True)],
                    epochs = EPOCH, verbose = 1)
        model.save_weights(f'{save_model_path}TPU-{MODEL_FILE_NAME}-{fold}.h5',overwrite=True)

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.43.68.202:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.43.68.202:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


#########################
### FOLD 4
#########################


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['embeddings.position_ids']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFRobertaModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.



Epoch 1: LearningRateScheduler setting learning rate to 3e-05.
Epoch 1/4
1014/1014 [==============================] - 536s 528ms/step - loss: 0.7142 - categorical_accuracy: 0.8402 - val_loss: 0.4284 - val_categorical_accuracy: 0.8881 - lr: 3.0000e-05

Epoch 2: LearningRateScheduler setting learning rate to 2e-05.
Epoch 2/4
1014/1014 [==============================] - 356s 351ms/step - loss: 0.3732 - categorical_accuracy: 0.9000 - val_loss: 0.3909 - val_categorical_accuracy: 0.8961 - lr: 2.0000e-05

Epoch 3: LearningRateScheduler setting learning rate to 1e-05.
Epoch 3/4
1014/1014 [==============================] - 356s 351ms/step - loss: 0.3015 - categorical_accuracy: 0.9166 - val_loss: 0.3828 - val_categorical_accuracy: 0.8984 - lr: 1.0000e-05

Epoch 4: LearningRateScheduler setting learning rate to 5e-06.
Epoch 4/4
1014/1014 [==============================] - 356s 351ms/step - loss: 0.2625 - categorical_accuracy: 0.9268 - val_loss: 0.3793 - val_categorical_accuracy: 0.8998 - lr: 5.0

# Check saved model

In [ ]:
model_list = [save_model_path+i for i in os.listdir(save_model_path)]
pred = 0
result_dic = {}

In [ ]:
model_list

['/content/drive/MyDrive/data_center_league/AI-based_industry_classification/Gu/[NAN]rob-large-no-duplicate-5fold/model/TPU-klue-roberta-large-0.h5',
 '/content/drive/MyDrive/data_center_league/AI-based_industry_classification/Gu/[NAN]rob-large-no-duplicate-5fold/model/TPU-klue-roberta-large-1.h5',
 '/content/drive/MyDrive/data_center_league/AI-based_industry_classification/Gu/[NAN]rob-large-no-duplicate-5fold/model/TPU-klue-roberta-large-2.h5',
 '/content/drive/MyDrive/data_center_league/AI-based_industry_classification/Gu/[NAN]rob-large-no-duplicate-5fold/model/TPU-klue-roberta-large-3.h5',
 '/content/drive/MyDrive/data_center_league/AI-based_industry_classification/Gu/[NAN]rob-large-no-duplicate-5fold/model/TPU-klue-roberta-large-4.h5']

# Test Data Preprocess

Test data preprocess is same above

In [ ]:
test_df = test[["text_obj","text_mthd","text_deal"]]
test_df.fillna("",inplace=True)
test_df.text_obj = test_df.text_obj.replace("[NAN]","")
test_df.text_mthd = test_df.text_mthd.replace("[NAN]","")
test_df.text_deal = test_df.text_deal.replace("[NAN]","")
test_df["sentence"] = test_df.text_obj + " " + test_df.text_mthd + " " + test_df.text_deal
test["sentence"] = test_df.sentence.progress_apply(lambda x: x.strip())
test_ds = Dataset.from_pandas(test[["sentence"]])
test_ds = test_ds.map(lambda x: tokenizer.encode_plus(x["sentence"],max_length=35,padding="max_length",truncation=True,return_tensors="np"))
test_ds = test_ds.remove_columns("sentence")
test_token = np.array(test_ds['input_ids']).squeeze(1)
test_attention = np.array(test_ds['attention_mask']).squeeze(1)
tf_test_ds = tf.data.Dataset.from_tensor_slices({"input_ids":test_ds["input_ids"],"attention_mask":test_ds["attention_mask"]})

100%|██████████| 100000/100000 [00:00<00:00, 873289.36it/s]


  0%|          | 0/100000 [00:00<?, ?ex/s]

# Model Ensemble

Before average ensemble with other model, it does average ensembles with 5fold.  

In [ ]:
%%time
pred = 0
for idx, path in enumerate(model_list):
    tf.tpu.experimental.initialize_tpu_system(tpu)
    print(f"********{idx} pred is on going **********")
    with strategy.scope():
        model = build_model()
        model.load_weights(path)
        pred += model.predict([test_token, test_attention],batch_size=100,)
result = pred / 5

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.43.68.202:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.43.68.202:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


********4 pred is on going **********


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['embeddings.position_ids']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFRobertaModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


CPU times: user 5min 24s, sys: 1min 42s, total: 7min 6s
Wall time: 25min 7s


# Final Submission

It loads HIBERT model`s logits.  
for average ensemble, convert 5fold roberta-large logits to torch tensor


In [ ]:
torch_result = torch.load('/content/drive/MyDrive/data_center_league/AI-based_industry_classification/kerry/data/skf_roberta_ensemble_logits.pt')
final_result = (torch_result + torch.tensor(result)) / 2

Predict The digit_3.  
digit_2, digit_1 are transformed from digit_3 by rule we found.  


In [ ]:
result = torch.argmax(final_result,axis=-1)
PRED = label_encoder.inverse_transform(result)
submission_df = test[["AI_id","digit_1","digit_2","digit_3","text_obj","text_mthd","text_deal"]]
submission_df.digit_3 = PRED
submission_df["digit_2"] = submission_df["digit_3"].progress_apply(lambda x: int(str(x)[:-1]))
dics = {}
for k in train.digit_1.unique():
    condition_list =  train[train.digit_1 == k].digit_2.unique()
    for j in condition_list:
        dics[j] = k
submission_df.digit_1 =  submission_df.digit_2.progress_apply(lambda x: dics[x])
submission_df.to_csv(f"/content/drive/MyDrive/data_center_league/AI-based_industry_classification/Gu/[NAN]rob-large-no-duplicate-5fold/result/ensemble_submission.csv",index=False)

100%|██████████| 100000/100000 [00:00<00:00, 816652.75it/s]
